In [1]:
import os
import random
import pandas as pd
import numpy as np
import csv
#import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split
#from google.colab import drive
import textwrap
import progressbar
import keras
#from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
import time
import datetime
import json

C:\Users\awast\anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
folder_path="C:\\Users\\awast\\Desktop\\NLP\\Data\\Data"

In [3]:
df = pd.read_csv("C:\\Users\\awast\\Desktop\\NLP\\Data\\Data\\ILDC_single\\ILDC_single\\ILDC_single.csv\\ILDC_single.csv")
train_set = df.query(" split=='train' ")
test_set = df.query(" split=='test' ")
validation_set = df.query(" split=='dev' ")

In [4]:
train_set

,text,label,split,name
0,"F. NARIMAN, J. Leave granted. In 2008, the Pu...",1,train,2019_890.txt
1,"S. THAKUR, J. Leave granted. These appeals ar...",0,train,2014_170.txt
2,"Markandey Katju, J. Leave granted. Heard lear...",1,train,2010_721.txt
3,"ALTAMAS KABIR,J. Leave granted. The question ...",1,train,2008_1460.txt
4,"CIVIL APPEAL NO. 598 OF 2007 K. MATHUR, J. Th...",1,train,2008_188.txt
...,...,...,...,...
5077,1999 1 SCR 104 The Judgments of the Court w...,1,train,1999_910.txt
5078,"Jagannadha Rao, J. This Interlocutory Applica...",1,train,1999_994.txt
5079,1999 4 Suppl. SCR 360 ORDER The following...,0,train,1999_1084.txt
5080,1999 Supp 5 SCR 186 The Judgment of the Cour...,1,train,1999_1147.txt


In [3]:
from transformers import PreTrainedModel, PreTrainedTokenizer, PretrainedConfig
from transformers import BertForSequenceClassification, BertTokenizer, BertConfig
from transformers import RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig
from transformers import XLNetForSequenceClassification, XLNetTokenizer, XLNetConfig
from transformers import XLMForSequenceClassification, XLMTokenizer, XLMConfig
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig

MODEL_CLASSES = {
    'bert': (BertForSequenceClassification, BertTokenizer, BertConfig),
    'xlnet': (XLNetForSequenceClassification, XLNetTokenizer, XLNetConfig),
    'xlm': (XLMForSequenceClassification, XLMTokenizer, XLMConfig),
    'roberta': (RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig),
    'distilbert': (DistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig)}

model_type = 'distilbert' ###--> CHANGE WHAT MODEL YOU WANT HERE!!! <--###
model_class, tokenizer_class, config_class = MODEL_CLASSES[model_type]
model_name = 'distilbert-base-uncased'

In [4]:
def att_masking(input_ids):
    attention_masks = []
    for sent in input_ids:
        att_mask = [int(token_id > 0) for token_id in sent]
        attention_masks.append(att_mask)
    return attention_masks

In [5]:
def grouped_input_ids(all_toks):
    splitted_toks = []
    l=0
    r=510
    while(l<len(all_toks)):
        splitted_toks.append(all_toks[l:min(r,len(all_toks))])
        l+=410
        r+=410

    CLS = tokenizer.cls_token
    SEP = tokenizer.sep_token
    e_sents = []
    for l_t in splitted_toks:
        l_t = l_t + [SEP] + [CLS]
        encoded_sent = tokenizer.convert_tokens_to_ids(l_t)
        e_sents.append(encoded_sent)

    e_sents = pad_sequences(e_sents, maxlen=512, value=0, dtype="long", padding="pre")
    att_masks = att_masking(e_sents)
    return e_sents, att_masks

In [6]:
def generate_np_files_for_training(dataf, tokenizer):
    all_input_ids, all_att_masks, all_labels = [], [], []
    for i in progressbar.progressbar(range(len(dataf['text']))):
        text = dataf['text'].iloc[i]
        toks = tokenizer.tokenize(text)
        if(len(toks) > 10000):
            toks = toks[len(toks)-10000:]

        splitted_input_ids, splitted_att_masks = grouped_input_ids(toks)
        doc_label = dataf['label'].iloc[i]
        print(len(splitted_input_ids))
        for j in range(len(splitted_input_ids)):
            all_input_ids.append(splitted_input_ids[j])
            all_att_masks.append(splitted_att_masks[j])
            all_labels.append(doc_label)

    return all_input_ids, all_att_masks, all_labels

In [7]:
from keras.utils import pad_sequences
#from keras.preprocessing.sequence import pad_sequences

In [10]:
from transformers import *
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)
train_input_ids, train_att_masks, train_labels = generate_np_files_for_training(train_set, tokenizer)

C:\Users\awast\anaconda3\envs\nlp\lib\site-packages\transformers\generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
C:\Users\awast\anaconda3\envs\nlp\lib\site-packages\transformers\generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
C:\Users\awast\anaconda3\envs\nlp\lib\site-packages\torchaudio\backend\utils.py:62: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
C:\Users\awast\anaconda3\envs\nlp\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated fil

15
16
10


  0% (5 of 5082) |                       | Elapsed Time: 0:00:00 ETA:   0:07:41

12
15
3


  0% (7 of 5082) |                       | Elapsed Time: 0:00:00 ETA:   0:07:06

...

8
5
7
18


 99% (5076 of 5082) |################### | Elapsed Time: 0:07:25 ETA:   0:00:00

4
5
8
5
2
14


100% (5082 of 5082) |####################| Elapsed Time: 0:07:25 Time:  0:07:25


13
12
4
4
3
2


In [7]:
def input_id_maker(dataf, tokenizer):
    input_ids = []
    lengths = []

    for i in progressbar.progressbar(range(len(dataf['text']))):
        sen = dataf['text'].iloc[i]
        sen = tokenizer.tokenize(sen, add_prefix_space=True)
        CLS = tokenizer.cls_token
        SEP = tokenizer.sep_token
        if(len(sen) > 510):
            sen = sen[len(sen)-510:]

    sen = sen + [SEP] + [CLS]
    encoded_sent = tokenizer.convert_tokens_to_ids(sen)
    input_ids.append(encoded_sent)
    lengths.append(len(encoded_sent))

    input_ids = pad_sequences(input_ids, maxlen=512, value=0, dtype="long", truncating="pre", padding="pre")
    return input_ids, lengths

In [12]:
validation_input_ids, validation_attention_masks, validation_labels = generate_np_files_for_training(validation_set, tokenizer)

  0% (3 of 994) |                        | Elapsed Time: 0:00:00 ETA:   0:01:49

19
7
22


  0% (6 of 994) |                        | Elapsed Time: 0:00:00 ETA:   0:01:36

8
12
17


  0% (8 of 994) |                        | Elapsed Time: 0:00:00 ETA:   0:01:37

...

6
3
7
6
13


 99% (989 of 994) |##################### | Elapsed Time: 0:01:19 ETA:   0:00:00

14
13
8
8
6
1


100% (994 of 994) |######################| Elapsed Time: 0:01:19 Time:  0:01:19


6
7
5


In [13]:
#validation_attention_masks = att_masking(validation_input_ids)
#validation_labels = validation_set['label'].to_numpy().astype('int')

In [14]:
train_inputs = train_input_ids
validation_inputs = validation_input_ids
train_masks = train_att_masks
validation_masks = validation_attention_masks

train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

C:\Users\awast\AppData\Local\Temp\ipykernel_9112\1819552211.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:233.)
  train_inputs = torch.tensor(train_inputs)


In [15]:
train_labels

tensor([1, 1, 1,  ..., 1, 0, 0])

In [16]:
print(train_inputs.shape)
print(train_masks.shape)
print(train_labels.shape)

torch.Size([57086, 512])
torch.Size([57086, 512])
torch.Size([57086])


In [17]:
print(validation_inputs.shape)
print(validation_masks.shape)
print(validation_labels.shape)

torch.Size([10623, 512])
torch.Size([10623, 512])
torch.Size([10623])


In [18]:
# batch_size = 4
# train_data = TensorDataset(train_inputs, train_masks, train_labels)
# train_sampler = RandomSampler(train_data)
# train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size = batch_size)
# validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
# validation_sampler = RandomSampler(validation_data)
# validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size = batch_size)


In [24]:
torch.save(train_data,'train_data.pt')
torch.save(validation_data,'validation_data.pt')

In [8]:
batch_size = 12
train_data = torch.load('train_data.pt')
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size = batch_size)
validation_data = torch.load('validation_data.pt')
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size = batch_size)


In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [10]:
lr = 2e-6
max_grad_norm = 1.0
epochs = 5
num_total_steps = len(train_dataloader)*epochs
num_warmup_steps = 1000
warmup_proportion = float(num_warmup_steps) / float(num_total_steps)  # 0.1
optimizer = AdamW(model.parameters(), lr=lr, correct_bias=True)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = num_warmup_steps, num_training_steps = num_total_steps)

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

seed_val = 21


np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

C:\Users\awast\anaconda3\envs\nlp\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [11]:
train_loss_values = []
train_accuracy = []
val_loss_values = []
val_accuracy = []

# For each epoch...
for epoch_i in range(0, 5):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    model.train()
    total_loss=0
    train_batch_accuracy = 0

    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}. : loss: {:} '.format(step, len(train_dataloader), total_loss/step))


        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

#         outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        logits = outputs[1]

        total_loss+=loss.item()

        loss.backward()

        batch_logits = logits
        logits = batch_logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        train_batch_accuracy = flat_accuracy(logits, label_ids)

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        if step%1000 == 0 and not step == 0:
            print("\nRunning Validation...")
            eval_loss, eval_accuracy = 0, 0
            nb_eval_steps, nb_eval_examples = 0, 0
            for batch in validation_dataloader:
                batch = tuple(t.to(device) for t in batch)
                b_input_ids, b_input_mask, b_labels = batch
                with torch.no_grad():        
                    outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)

                loss = outputs[0]
                logits = outputs[1]
    
                logits = logits.detach().cpu().numpy()
                label_ids = b_labels.to('cpu').numpy()
        
                tmp_eval_accuracy = flat_accuracy(logits, label_ids)
                eval_accuracy += tmp_eval_accuracy

                eval_loss+=loss
 
                nb_eval_steps += 1

            val_accuracy.append(eval_accuracy/nb_eval_steps)
            val_loss_values.append(eval_loss/nb_eval_steps)

            print('Validation loss: {:} : Validation accuracy: {:}'.format(val_loss_values[-1], val_accuracy[-1]))

        
    train_loss_values.append(total_loss/len(train_dataloader))
    train_accuracy.append(train_batch_accuracy/len(train_dataloader))


print("Training complete!")

======== Epoch 1 / 5 ========
Training...
  Batch    40  of  4,758. : loss: 0.7103865042328834 
  Batch    80  of  4,758. : loss: 0.7114255972206592 
  Batch   120  of  4,758. : loss: 0.7066099748015404 
  Batch   160  of  4,758. : loss: 0.7053252439945936 
  Batch   200  of  4,758. : loss: 0.7037649601697922 
  Batch   240  of  4,758. : loss: 0.7013995309670766 
  Batch   280  of  4,758. : loss: 0.6986948815839631 
  Batch   320  of  4,758. : loss: 0.6951765870675445 
  Batch   360  of  4,758. : loss: 0.6930226541227764 
  Batch   400  of  4,758. : loss: 0.6905590541660785 
  Batch   440  of  4,758. : loss: 0.6894499330358071 
  Batch   480  of  4,758. : loss: 0.6878965886930625 
  Batch   520  of  4,758. : loss: 0.687594986305787 
  Batch   560  of  4,758. : loss: 0.686935789670263 
  Batch   600  of  4,758. : loss: 0.6851780196030934 
  Batch   640  of  4,758. : loss: 0.683341774251312 
  Batch   680  of  4,758. : loss: 0.6829154654460795 
  Batch   720  of  4,758. : loss: 0.6815176

Validation loss: 0.6693061590194702 : Validation accuracy: 0.5916102332580891
  Batch 1,040  of  4,758. : loss: 0.6115570759257445 
  Batch 1,080  of  4,758. : loss: 0.6108365739937182 
  Batch 1,120  of  4,758. : loss: 0.6109788685238787 
  Batch 1,160  of  4,758. : loss: 0.6108764881717748 
  Batch 1,200  of  4,758. : loss: 0.6103899640093247 
  Batch 1,240  of  4,758. : loss: 0.6094765636709428 
  Batch 1,280  of  4,758. : loss: 0.6089832019992173 
  Batch 1,320  of  4,758. : loss: 0.6086328096236243 
  Batch 1,360  of  4,758. : loss: 0.608788377923124 
  Batch 1,400  of  4,758. : loss: 0.6089625717060907 
  Batch 1,440  of  4,758. : loss: 0.6082641812455323 
  Batch 1,480  of  4,758. : loss: 0.6079012133986563 
  Batch 1,520  of  4,758. : loss: 0.6080278107798414 
  Batch 1,560  of  4,758. : loss: 0.6077432635502937 
  Batch 1,600  of  4,758. : loss: 0.6075710866600275 
  Batch 1,640  of  4,758. : loss: 0.6072855627936561 
  Batch 1,680  of  4,758. : loss: 0.6076967767484132 
  Bat


Running Validation...
Validation loss: 0.672001302242279 : Validation accuracy: 0.5993227990970665
  Batch 2,040  of  4,758. : loss: 0.5749396981269705 
  Batch 2,080  of  4,758. : loss: 0.574808851772776 
  Batch 2,120  of  4,758. : loss: 0.5741029550022674 
  Batch 2,160  of  4,758. : loss: 0.5742748746924379 
  Batch 2,200  of  4,758. : loss: 0.5743740230189128 
  Batch 2,240  of  4,758. : loss: 0.5739998806600592 
  Batch 2,280  of  4,758. : loss: 0.5742904520413855 
  Batch 2,320  of  4,758. : loss: 0.5742168768919234 
  Batch 2,360  of  4,758. : loss: 0.574307091073212 
  Batch 2,400  of  4,758. : loss: 0.5741622364831468 
  Batch 2,440  of  4,758. : loss: 0.5739582161983994 
  Batch 2,480  of  4,758. : loss: 0.5734971076430332 
  Batch 2,520  of  4,758. : loss: 0.5736105526427902 
  Batch 2,560  of  4,758. : loss: 0.5743550601124298 
  Batch 2,600  of  4,758. : loss: 0.5740498781490785 
  Batch 2,640  of  4,758. : loss: 0.5740243268904812 
  Batch 2,680  of  4,758. : loss: 0.57

  Batch 3,000  of  4,758. : loss: 0.5591916798502207 

Running Validation...
Validation loss: 0.6890211701393127 : Validation accuracy: 0.6030850263355912
  Batch 3,040  of  4,758. : loss: 0.5591655582532679 
  Batch 3,080  of  4,758. : loss: 0.559568474596584 
  Batch 3,120  of  4,758. : loss: 0.5598219610846196 
  Batch 3,160  of  4,758. : loss: 0.5598374812097489 
  Batch 3,200  of  4,758. : loss: 0.5600222019292415 
  Batch 3,240  of  4,758. : loss: 0.5598203929394115 
  Batch 3,280  of  4,758. : loss: 0.5594309356427047 
  Batch 3,320  of  4,758. : loss: 0.558986503577017 
  Batch 3,360  of  4,758. : loss: 0.5591337604714292 
  Batch 3,400  of  4,758. : loss: 0.5587656347190633 
  Batch 3,440  of  4,758. : loss: 0.5589257377611343 
  Batch 3,480  of  4,758. : loss: 0.5589994253098279 
  Batch 3,520  of  4,758. : loss: 0.5587295857258141 
  Batch 3,560  of  4,758. : loss: 0.5588405153389727 
  Batch 3,600  of  4,758. : loss: 0.5584646176132891 
  Batch 3,640  of  4,758. : loss: 0.5

  Batch 3,960  of  4,758. : loss: 0.5436857820196886 
  Batch 4,000  of  4,758. : loss: 0.5442256056703627 

Running Validation...
Validation loss: 0.6910686492919922 : Validation accuracy: 0.5982881866064712
  Batch 4,040  of  4,758. : loss: 0.5444123638922921 
  Batch 4,080  of  4,758. : loss: 0.5443314645695043 
  Batch 4,120  of  4,758. : loss: 0.5446658272947212 
  Batch 4,160  of  4,758. : loss: 0.5443023195848442 
  Batch 4,200  of  4,758. : loss: 0.5443101621880417 
  Batch 4,240  of  4,758. : loss: 0.544468887881288 
  Batch 4,280  of  4,758. : loss: 0.544892247097793 
  Batch 4,320  of  4,758. : loss: 0.5451698645535442 
  Batch 4,360  of  4,758. : loss: 0.5449633685000446 
  Batch 4,400  of  4,758. : loss: 0.5447019556469538 
  Batch 4,440  of  4,758. : loss: 0.5445011858501144 
  Batch 4,480  of  4,758. : loss: 0.5448110587196424 
  Batch 4,520  of  4,758. : loss: 0.5449062454918053 
  Batch 4,560  of  4,758. : loss: 0.5451137457467746 
  Batch 4,600  of  4,758. : loss: 0.5

In [12]:
output_dir = "distilbert_single" # path to which fine tuned model is to be saved

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))

# Copy the model files to a directory in your Google Drive.
# !cp -r ./mini_XLNet/ "/content/Drive/My Drive/mini_XLNet/"

Saving model to distilbert_single


NameError: name 'tokenizer' is not defined

In [13]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)
tokenizer.save_pretrained(output_dir)


('distilbert_single\\tokenizer_config.json',
 'distilbert_single\\special_tokens_map.json',
 'distilbert_single\\vocab.txt',
 'distilbert_single\\added_tokens.json')

In [14]:
df = pd.read_csv("C:\\Users\\awast\\Desktop\\NLP\\Data\\Data\\ILDC_single\\ILDC_single\\ILDC_single.csv\\ILDC_single.csv")
train_set = df.query(" split=='train' ")
test_set = df.query(" split=='test' ")
validation_set = df.query(" split=='dev' ")

In [15]:
# output_dir = "/content/Drive/My Drive/mini_XLNet/"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DistilBertForSequenceClassification.from_pretrained(output_dir, output_hidden_states=True)
tokenizer = DistilBertTokenizer.from_pretrained(output_dir)
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [81]:
def att_masking(input_ids):
    attention_masks = []
    for sent in input_ids:
        att_mask = [int(token_id > 0) for token_id in sent]
        attention_masks.append(att_mask)
    return attention_masks

In [82]:
def grouped_input_ids(all_toks):
    splitted_toks = []
    l=0
    r=510
    while(l<len(all_toks)):
        splitted_toks.append(all_toks[l:min(r,len(all_toks))])
        l+=410
        r+=410

    CLS = tokenizer.cls_token
    SEP = tokenizer.sep_token
    e_sents = []
    for l_t in splitted_toks:
        l_t = l_t + [SEP] + [CLS]
        encoded_sent = tokenizer.convert_tokens_to_ids(l_t)
        e_sents.append(encoded_sent)

    e_sents = pad_sequences(e_sents, maxlen=512, value=0, dtype="long", padding="pre")
    att_masks = att_masking(e_sents)
    return e_sents, att_masks

In [83]:
def get_output_for_one_vec1(input_id, att_mask):
    input_ids = torch.tensor(input_id)
    att_masks = torch.tensor(att_mask)
    input_ids = input_ids.unsqueeze(0)
    att_masks = att_masks.unsqueeze(0)
    model.eval()
    input_ids = input_ids.to(device)
    att_masks = att_masks.to(device)
    with torch.no_grad():
        logits, encoded_layers = model(input_ids=input_ids, attention_mask=att_masks)
        outputs=model(input_ids=input_ids, attention_mask=att_masks)
#         print(outputs)

    vec = outputs.hidden_states[6][0][-1]
    
    vec = vec.detach().cpu().numpy()
    
#     input_ids = torch.tensor(input_id)
#     att_masks = torch.tensor(att_mask)
#     input_ids = input_ids.unsqueeze(0)
#     att_masks = att_masks.unsqueeze(0)
#     model.eval()
#     input_ids = input_ids.to(device)
#     att_masks = att_masks.to(device)
#     with torch.no_grad():
#         # logits, encoded_layers = model(input_ids=input_ids, token_type_ids=None, attention_mask=att_masks)
        
#         outputs = model(input_ids=input_ids, attention_mask=att_masks)
#         print(outputs.hidden_states[6])
                
        
#         #return outputs

# #     input_ids=input_ids.detach().cpu()
# #     att_masks=att_masks.detach().cpu()
    
#     vec = outputs.mems[11][0][-1]
#     #print(vec.shape)
#     vec = vec.detach().cpu().numpy()
    return vec
    

In [89]:
def generate_np_files_for_emb(dataf, tokenizer):
    all_docs = []
    for i in progressbar.progressbar(range(len(dataf['text']))):
        text = dataf['text'].iloc[i]
        toks = tokenizer.tokenize(text, add_prefix_space=True)
        if(len(toks) > 10000):
            toks = toks[len(toks)-10000:]

        splitted_input_ids, splitted_att_masks = grouped_input_ids(toks)

        vecs = []
        for index,ii in enumerate(splitted_input_ids):
            vecs.append(get_output_for_one_vec1(ii, splitted_att_masks[index]))

        one_doc = np.asarray(vecs)
        all_docs.append(one_doc)

    all_docs = np.asarray(all_docs)
    return all_docs

In [90]:
vecs_dev = generate_np_files_for_emb(validation_set, tokenizer)
np.save("Distilbert_single/distil_dev.npy", vecs_dev)

  0% (0 of 994) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--Keyword arguments {'add_prefix_space': True} not recognized.
  0% (1 of 994) |                        | Elapsed Time: 0:00:02 ETA:   0:39:24Keyword arguments {'add_prefix_space': True} not recognized.
  0% (2 of 994) |                        | Elapsed Time: 0:00:02 ETA:   0:22:52Keyword arguments {'add_prefix_space': True} not recognized.
  0% (3 of 994) |                        | Elapsed Time: 0:00:04 ETA:   0:22:25Keyword arguments {'add_prefix_space': True} not recognized.
  0% (4 of 994) |                        | Elapsed Time: 0:00:04 ETA:   0:11:43Keyword arguments {'add_prefix_space': True} not recognized.
  0% (5 of 994) |                        | Elapsed Time: 0:00:05 ETA:   0:13:21Keyword arguments {'add_prefix_space': True} not recognized.
  0% (6 of 994) |                        | Elapsed Time: 0:00:06 ETA:   0:11:25Keyword arguments {'add_prefix_space': True} not recognized.
  0% (7 of 994) |   

 11% (116 of 994) |##                    | Elapsed Time: 0:01:14 ETA:   0:07:24Keyword arguments {'add_prefix_space': True} not recognized.
 11% (117 of 994) |##                    | Elapsed Time: 0:01:14 ETA:   0:06:34Keyword arguments {'add_prefix_space': True} not recognized.
 11% (118 of 994) |##                    | Elapsed Time: 0:01:14 ETA:   0:07:27Keyword arguments {'add_prefix_space': True} not recognized.
 11% (119 of 994) |##                    | Elapsed Time: 0:01:15 ETA:   0:08:23Keyword arguments {'add_prefix_space': True} not recognized.
 12% (120 of 994) |##                    | Elapsed Time: 0:01:16 ETA:   0:10:06Keyword arguments {'add_prefix_space': True} not recognized.
 12% (121 of 994) |##                    | Elapsed Time: 0:01:17 ETA:   0:12:58Keyword arguments {'add_prefix_space': True} not recognized.
 12% (122 of 994) |##                    | Elapsed Time: 0:01:18 ETA:   0:10:37Keyword arguments {'add_prefix_space': True} not recognized.
 12% (123 of 994) |#

 23% (234 of 994) |#####                 | Elapsed Time: 0:02:33 ETA:   0:06:54Keyword arguments {'add_prefix_space': True} not recognized.
 23% (235 of 994) |#####                 | Elapsed Time: 0:02:34 ETA:   0:09:38Keyword arguments {'add_prefix_space': True} not recognized.
 23% (236 of 994) |#####                 | Elapsed Time: 0:02:35 ETA:   0:12:53Keyword arguments {'add_prefix_space': True} not recognized.
 23% (237 of 994) |#####                 | Elapsed Time: 0:02:36 ETA:   0:16:05Keyword arguments {'add_prefix_space': True} not recognized.
 23% (238 of 994) |#####                 | Elapsed Time: 0:02:37 ETA:   0:12:53Keyword arguments {'add_prefix_space': True} not recognized.
 24% (239 of 994) |#####                 | Elapsed Time: 0:02:38 ETA:   0:10:55Keyword arguments {'add_prefix_space': True} not recognized.
 24% (240 of 994) |#####                 | Elapsed Time: 0:02:39 ETA:   0:09:10Keyword arguments {'add_prefix_space': True} not recognized.
 24% (241 of 994) |#

 35% (351 of 994) |#######               | Elapsed Time: 0:03:54 ETA:   0:07:32Keyword arguments {'add_prefix_space': True} not recognized.
 35% (352 of 994) |#######               | Elapsed Time: 0:03:55 ETA:   0:12:37Keyword arguments {'add_prefix_space': True} not recognized.
 35% (353 of 994) |#######               | Elapsed Time: 0:03:56 ETA:   0:10:06Keyword arguments {'add_prefix_space': True} not recognized.
 35% (354 of 994) |#######               | Elapsed Time: 0:03:57 ETA:   0:09:42Keyword arguments {'add_prefix_space': True} not recognized.
 35% (355 of 994) |#######               | Elapsed Time: 0:03:57 ETA:   0:08:46Keyword arguments {'add_prefix_space': True} not recognized.
 35% (356 of 994) |#######               | Elapsed Time: 0:03:58 ETA:   0:07:24Keyword arguments {'add_prefix_space': True} not recognized.
 35% (357 of 994) |#######               | Elapsed Time: 0:03:58 ETA:   0:06:42Keyword arguments {'add_prefix_space': True} not recognized.
 36% (358 of 994) |#

 46% (467 of 994) |##########            | Elapsed Time: 0:05:09 ETA:   0:10:38Keyword arguments {'add_prefix_space': True} not recognized.
 47% (468 of 994) |##########            | Elapsed Time: 0:05:11 ETA:   0:10:12Keyword arguments {'add_prefix_space': True} not recognized.
 47% (469 of 994) |##########            | Elapsed Time: 0:05:12 ETA:   0:11:00Keyword arguments {'add_prefix_space': True} not recognized.
 47% (470 of 994) |##########            | Elapsed Time: 0:05:12 ETA:   0:08:06Keyword arguments {'add_prefix_space': True} not recognized.
 47% (471 of 994) |##########            | Elapsed Time: 0:05:12 ETA:   0:06:19Keyword arguments {'add_prefix_space': True} not recognized.
 47% (472 of 994) |##########            | Elapsed Time: 0:05:14 ETA:   0:05:59Keyword arguments {'add_prefix_space': True} not recognized.
 47% (473 of 994) |##########            | Elapsed Time: 0:05:15 ETA:   0:13:10Keyword arguments {'add_prefix_space': True} not recognized.
 47% (474 of 994) |#

 58% (583 of 994) |############          | Elapsed Time: 0:06:23 ETA:   0:07:13Keyword arguments {'add_prefix_space': True} not recognized.
Keyword arguments {'add_prefix_space': True} not recognized.
 58% (585 of 994) |############          | Elapsed Time: 0:06:23 ETA:   0:04:33Keyword arguments {'add_prefix_space': True} not recognized.
 58% (586 of 994) |############          | Elapsed Time: 0:06:24 ETA:   0:05:00Keyword arguments {'add_prefix_space': True} not recognized.
 59% (587 of 994) |############          | Elapsed Time: 0:06:25 ETA:   0:03:39Keyword arguments {'add_prefix_space': True} not recognized.
 59% (588 of 994) |#############         | Elapsed Time: 0:06:25 ETA:   0:03:09Keyword arguments {'add_prefix_space': True} not recognized.
 59% (589 of 994) |#############         | Elapsed Time: 0:06:27 ETA:   0:05:07Keyword arguments {'add_prefix_space': True} not recognized.
 59% (590 of 994) |#############         | Elapsed Time: 0:06:27 ETA:   0:07:11Keyword arguments {'

 70% (700 of 994) |###############       | Elapsed Time: 0:07:46 ETA:   0:03:03Keyword arguments {'add_prefix_space': True} not recognized.
 70% (701 of 994) |###############       | Elapsed Time: 0:07:48 ETA:   0:05:45Keyword arguments {'add_prefix_space': True} not recognized.
 70% (702 of 994) |###############       | Elapsed Time: 0:07:48 ETA:   0:04:23Keyword arguments {'add_prefix_space': True} not recognized.
 70% (703 of 994) |###############       | Elapsed Time: 0:07:48 ETA:   0:03:50Keyword arguments {'add_prefix_space': True} not recognized.
 70% (704 of 994) |###############       | Elapsed Time: 0:07:49 ETA:   0:03:31Keyword arguments {'add_prefix_space': True} not recognized.
 70% (705 of 994) |###############       | Elapsed Time: 0:07:49 ETA:   0:03:09Keyword arguments {'add_prefix_space': True} not recognized.
 71% (706 of 994) |###############       | Elapsed Time: 0:07:50 ETA:   0:02:51Keyword arguments {'add_prefix_space': True} not recognized.
 71% (707 of 994) |#

 82% (816 of 994) |##################    | Elapsed Time: 0:09:03 ETA:   0:01:29Keyword arguments {'add_prefix_space': True} not recognized.
 82% (817 of 994) |##################    | Elapsed Time: 0:09:04 ETA:   0:01:40Keyword arguments {'add_prefix_space': True} not recognized.
 82% (818 of 994) |##################    | Elapsed Time: 0:09:05 ETA:   0:02:04Keyword arguments {'add_prefix_space': True} not recognized.
 82% (819 of 994) |##################    | Elapsed Time: 0:09:06 ETA:   0:04:08Keyword arguments {'add_prefix_space': True} not recognized.
 82% (820 of 994) |##################    | Elapsed Time: 0:09:07 ETA:   0:03:24Keyword arguments {'add_prefix_space': True} not recognized.
 82% (821 of 994) |##################    | Elapsed Time: 0:09:08 ETA:   0:02:54Keyword arguments {'add_prefix_space': True} not recognized.
 82% (822 of 994) |##################    | Elapsed Time: 0:09:08 ETA:   0:02:23Keyword arguments {'add_prefix_space': True} not recognized.
 82% (823 of 994) |#

 93% (932 of 994) |####################  | Elapsed Time: 0:10:23 ETA:   0:00:44Keyword arguments {'add_prefix_space': True} not recognized.
 93% (933 of 994) |####################  | Elapsed Time: 0:10:23 ETA:   0:00:28Keyword arguments {'add_prefix_space': True} not recognized.
 93% (934 of 994) |####################  | Elapsed Time: 0:10:24 ETA:   0:00:30Keyword arguments {'add_prefix_space': True} not recognized.
 94% (935 of 994) |####################  | Elapsed Time: 0:10:24 ETA:   0:00:30Keyword arguments {'add_prefix_space': True} not recognized.
 94% (936 of 994) |####################  | Elapsed Time: 0:10:25 ETA:   0:00:28Keyword arguments {'add_prefix_space': True} not recognized.
 94% (937 of 994) |####################  | Elapsed Time: 0:10:25 ETA:   0:00:25Keyword arguments {'add_prefix_space': True} not recognized.
 94% (938 of 994) |####################  | Elapsed Time: 0:10:26 ETA:   0:00:31Keyword arguments {'add_prefix_space': True} not recognized.
 94% (939 of 994) |#

In [91]:
vecs_train = generate_np_files_for_emb(train_set, tokenizer)
np.save("Distilbert_single/distil_train.npy", vecs_train)

  0% (0 of 5082) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--Keyword arguments {'add_prefix_space': True} not recognized.
  0% (1 of 5082) |                       | Elapsed Time: 0:00:00 ETA:   1:16:07Keyword arguments {'add_prefix_space': True} not recognized.
  0% (2 of 5082) |                       | Elapsed Time: 0:00:01 ETA:   1:20:23Keyword arguments {'add_prefix_space': True} not recognized.
  0% (3 of 5082) |                       | Elapsed Time: 0:00:02 ETA:   1:10:24Keyword arguments {'add_prefix_space': True} not recognized.
  0% (4 of 5082) |                       | Elapsed Time: 0:00:03 ETA:   1:05:33Keyword arguments {'add_prefix_space': True} not recognized.
  0% (5 of 5082) |                       | Elapsed Time: 0:00:04 ETA:   1:02:51Keyword arguments {'add_prefix_space': True} not recognized.
  0% (6 of 5082) |                       | Elapsed Time: 0:00:04 ETA:   0:51:33Keyword arguments {'add_prefix_space': True} not recognized.
  0% (7 of 5082) |  

  2% (117 of 5082) |                     | Elapsed Time: 0:01:43 ETA:   1:04:24Keyword arguments {'add_prefix_space': True} not recognized.
  2% (118 of 5082) |                     | Elapsed Time: 0:01:43 ETA:   0:50:40Keyword arguments {'add_prefix_space': True} not recognized.
  2% (119 of 5082) |                     | Elapsed Time: 0:01:44 ETA:   0:50:18Keyword arguments {'add_prefix_space': True} not recognized.
  2% (120 of 5082) |                     | Elapsed Time: 0:01:44 ETA:   0:44:28Keyword arguments {'add_prefix_space': True} not recognized.
  2% (121 of 5082) |                     | Elapsed Time: 0:01:46 ETA:   1:00:15Keyword arguments {'add_prefix_space': True} not recognized.
  2% (122 of 5082) |                     | Elapsed Time: 0:01:46 ETA:   0:59:59Keyword arguments {'add_prefix_space': True} not recognized.
  2% (123 of 5082) |                     | Elapsed Time: 0:01:47 ETA:   1:03:48Keyword arguments {'add_prefix_space': True} not recognized.
  2% (124 of 5082) |

  4% (234 of 5082) |                     | Elapsed Time: 0:03:17 ETA:   1:24:18Keyword arguments {'add_prefix_space': True} not recognized.
  4% (235 of 5082) |                     | Elapsed Time: 0:03:17 ETA:   1:15:51Keyword arguments {'add_prefix_space': True} not recognized.
  4% (236 of 5082) |                     | Elapsed Time: 0:03:20 ETA:   3:46:13Keyword arguments {'add_prefix_space': True} not recognized.
  4% (237 of 5082) |                     | Elapsed Time: 0:03:21 ETA:   2:30:45Keyword arguments {'add_prefix_space': True} not recognized.
  4% (238 of 5082) |                     | Elapsed Time: 0:03:22 ETA:   1:52:26Keyword arguments {'add_prefix_space': True} not recognized.
  4% (239 of 5082) |                     | Elapsed Time: 0:03:23 ETA:   1:24:41Keyword arguments {'add_prefix_space': True} not recognized.
  4% (240 of 5082) |                     | Elapsed Time: 0:03:25 ETA:   2:22:50Keyword arguments {'add_prefix_space': True} not recognized.
  4% (241 of 5082) |

  6% (350 of 5082) |#                    | Elapsed Time: 0:05:02 ETA:   0:52:57Keyword arguments {'add_prefix_space': True} not recognized.
  6% (351 of 5082) |#                    | Elapsed Time: 0:05:03 ETA:   1:33:48Keyword arguments {'add_prefix_space': True} not recognized.
  6% (352 of 5082) |#                    | Elapsed Time: 0:05:04 ETA:   1:26:58Keyword arguments {'add_prefix_space': True} not recognized.
  6% (353 of 5082) |#                    | Elapsed Time: 0:05:04 ETA:   1:06:18Keyword arguments {'add_prefix_space': True} not recognized.
  6% (354 of 5082) |#                    | Elapsed Time: 0:05:05 ETA:   0:57:43Keyword arguments {'add_prefix_space': True} not recognized.
  6% (355 of 5082) |#                    | Elapsed Time: 0:05:06 ETA:   0:52:36Keyword arguments {'add_prefix_space': True} not recognized.
  7% (356 of 5082) |#                    | Elapsed Time: 0:05:07 ETA:   1:23:40Keyword arguments {'add_prefix_space': True} not recognized.
  7% (357 of 5082) |

  9% (467 of 5082) |#                    | Elapsed Time: 0:06:12 ETA:   0:30:16Keyword arguments {'add_prefix_space': True} not recognized.
  9% (468 of 5082) |#                    | Elapsed Time: 0:06:13 ETA:   0:28:44Keyword arguments {'add_prefix_space': True} not recognized.
  9% (469 of 5082) |#                    | Elapsed Time: 0:06:13 ETA:   0:28:12Keyword arguments {'add_prefix_space': True} not recognized.
  9% (470 of 5082) |#                    | Elapsed Time: 0:06:14 ETA:   0:27:21Keyword arguments {'add_prefix_space': True} not recognized.
  9% (471 of 5082) |#                    | Elapsed Time: 0:06:14 ETA:   0:28:53Keyword arguments {'add_prefix_space': True} not recognized.
  9% (472 of 5082) |#                    | Elapsed Time: 0:06:15 ETA:   0:43:02Keyword arguments {'add_prefix_space': True} not recognized.
  9% (473 of 5082) |#                    | Elapsed Time: 0:06:16 ETA:   0:41:49Keyword arguments {'add_prefix_space': True} not recognized.
  9% (474 of 5082) |

...

 90% (4596 of 5082) |##################  | Elapsed Time: 0:54:03 ETA:   0:05:12Keyword arguments {'add_prefix_space': True} not recognized.
 90% (4597 of 5082) |##################  | Elapsed Time: 0:54:04 ETA:   0:06:17Keyword arguments {'add_prefix_space': True} not recognized.
 90% (4598 of 5082) |##################  | Elapsed Time: 0:54:04 ETA:   0:06:02Keyword arguments {'add_prefix_space': True} not recognized.
 90% (4599 of 5082) |##################  | Elapsed Time: 0:54:05 ETA:   0:04:57Keyword arguments {'add_prefix_space': True} not recognized.
 90% (4600 of 5082) |##################  | Elapsed Time: 0:54:06 ETA:   0:04:38Keyword arguments {'add_prefix_space': True} not recognized.
 90% (4601 of 5082) |##################  | Elapsed Time: 0:54:07 ETA:   0:08:46Keyword arguments {'add_prefix_space': True} not recognized.
 90% (4602 of 5082) |##################  | Elapsed Time: 0:54:08 ETA:   0:09:44Keyword arguments {'add_prefix_space': True} not recognized.
 90% (4603 of 5082) 

 92% (4712 of 5082) |##################  | Elapsed Time: 0:55:17 ETA:   0:04:14Keyword arguments {'add_prefix_space': True} not recognized.
 92% (4713 of 5082) |##################  | Elapsed Time: 0:55:18 ETA:   0:07:44Keyword arguments {'add_prefix_space': True} not recognized.
 92% (4714 of 5082) |##################  | Elapsed Time: 0:55:20 ETA:   0:07:38Keyword arguments {'add_prefix_space': True} not recognized.
 92% (4715 of 5082) |##################  | Elapsed Time: 0:55:21 ETA:   0:08:06Keyword arguments {'add_prefix_space': True} not recognized.
 92% (4716 of 5082) |##################  | Elapsed Time: 0:55:21 ETA:   0:05:58Keyword arguments {'add_prefix_space': True} not recognized.
 92% (4717 of 5082) |##################  | Elapsed Time: 0:55:22 ETA:   0:04:04Keyword arguments {'add_prefix_space': True} not recognized.
 92% (4718 of 5082) |##################  | Elapsed Time: 0:55:22 ETA:   0:03:42Keyword arguments {'add_prefix_space': True} not recognized.
 92% (4719 of 5082) 

 95% (4829 of 5082) |################### | Elapsed Time: 0:56:31 ETA:   0:02:22Keyword arguments {'add_prefix_space': True} not recognized.
 95% (4830 of 5082) |################### | Elapsed Time: 0:56:32 ETA:   0:02:34Keyword arguments {'add_prefix_space': True} not recognized.
 95% (4831 of 5082) |################### | Elapsed Time: 0:56:32 ETA:   0:02:17Keyword arguments {'add_prefix_space': True} not recognized.
 95% (4832 of 5082) |################### | Elapsed Time: 0:56:32 ETA:   0:02:00Keyword arguments {'add_prefix_space': True} not recognized.
 95% (4833 of 5082) |################### | Elapsed Time: 0:56:34 ETA:   0:02:29Keyword arguments {'add_prefix_space': True} not recognized.
 95% (4834 of 5082) |################### | Elapsed Time: 0:56:35 ETA:   0:05:44Keyword arguments {'add_prefix_space': True} not recognized.
 95% (4835 of 5082) |################### | Elapsed Time: 0:56:36 ETA:   0:04:57Keyword arguments {'add_prefix_space': True} not recognized.
 95% (4836 of 5082) 

 97% (4946 of 5082) |################### | Elapsed Time: 0:57:56 ETA:   0:03:00Keyword arguments {'add_prefix_space': True} not recognized.
 97% (4947 of 5082) |################### | Elapsed Time: 0:57:57 ETA:   0:02:20Keyword arguments {'add_prefix_space': True} not recognized.
 97% (4948 of 5082) |################### | Elapsed Time: 0:57:57 ETA:   0:01:51Keyword arguments {'add_prefix_space': True} not recognized.
 97% (4949 of 5082) |################### | Elapsed Time: 0:57:57 ETA:   0:01:06Keyword arguments {'add_prefix_space': True} not recognized.
 97% (4950 of 5082) |################### | Elapsed Time: 0:57:58 ETA:   0:01:16Keyword arguments {'add_prefix_space': True} not recognized.
 97% (4951 of 5082) |################### | Elapsed Time: 0:58:00 ETA:   0:02:37Keyword arguments {'add_prefix_space': True} not recognized.
 97% (4952 of 5082) |################### | Elapsed Time: 0:58:00 ETA:   0:02:32Keyword arguments {'add_prefix_space': True} not recognized.
 97% (4953 of 5082) 

 99% (5064 of 5082) |################### | Elapsed Time: 0:59:04 ETA:   0:00:18Keyword arguments {'add_prefix_space': True} not recognized.
 99% (5065 of 5082) |################### | Elapsed Time: 0:59:04 ETA:   0:00:12Keyword arguments {'add_prefix_space': True} not recognized.
 99% (5066 of 5082) |################### | Elapsed Time: 0:59:05 ETA:   0:00:12Keyword arguments {'add_prefix_space': True} not recognized.
 99% (5067 of 5082) |################### | Elapsed Time: 0:59:05 ETA:   0:00:08Keyword arguments {'add_prefix_space': True} not recognized.
 99% (5068 of 5082) |################### | Elapsed Time: 0:59:05 ETA:   0:00:07Keyword arguments {'add_prefix_space': True} not recognized.
 99% (5069 of 5082) |################### | Elapsed Time: 0:59:06 ETA:   0:00:06Keyword arguments {'add_prefix_space': True} not recognized.
 99% (5070 of 5082) |################### | Elapsed Time: 0:59:07 ETA:   0:00:07Keyword arguments {'add_prefix_space': True} not recognized.
 99% (5071 of 5082) 

In [92]:
vecs_test = generate_np_files_for_emb(test_set, tokenizer)
np.save("Distilbert_single/distil_test.npy", vecs_test)

  0% (0 of 1517) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--Keyword arguments {'add_prefix_space': True} not recognized.
  0% (1 of 1517) |                       | Elapsed Time: 0:00:00 ETA:   0:11:32Keyword arguments {'add_prefix_space': True} not recognized.
  0% (2 of 1517) |                       | Elapsed Time: 0:00:00 ETA:   0:09:19Keyword arguments {'add_prefix_space': True} not recognized.
  0% (3 of 1517) |                       | Elapsed Time: 0:00:01 ETA:   0:10:57Keyword arguments {'add_prefix_space': True} not recognized.
  0% (4 of 1517) |                       | Elapsed Time: 0:00:02 ETA:   0:12:45Keyword arguments {'add_prefix_space': True} not recognized.
  0% (5 of 1517) |                       | Elapsed Time: 0:00:02 ETA:   0:16:54Keyword arguments {'add_prefix_space': True} not recognized.
  0% (6 of 1517) |                       | Elapsed Time: 0:00:03 ETA:   0:18:43Keyword arguments {'add_prefix_space': True} not recognized.
  0% (7 of 1517) |  

  7% (117 of 1517) |#                    | Elapsed Time: 0:01:30 ETA:   0:17:34Keyword arguments {'add_prefix_space': True} not recognized.
  7% (118 of 1517) |#                    | Elapsed Time: 0:01:32 ETA:   0:27:55Keyword arguments {'add_prefix_space': True} not recognized.
  7% (119 of 1517) |#                    | Elapsed Time: 0:01:32 ETA:   0:30:50Keyword arguments {'add_prefix_space': True} not recognized.
  7% (120 of 1517) |#                    | Elapsed Time: 0:01:34 ETA:   0:23:30Keyword arguments {'add_prefix_space': True} not recognized.
  7% (121 of 1517) |#                    | Elapsed Time: 0:01:35 ETA:   0:26:08Keyword arguments {'add_prefix_space': True} not recognized.
  8% (122 of 1517) |#                    | Elapsed Time: 0:01:35 ETA:   0:20:58Keyword arguments {'add_prefix_space': True} not recognized.
  8% (123 of 1517) |#                    | Elapsed Time: 0:01:36 ETA:   0:17:03Keyword arguments {'add_prefix_space': True} not recognized.
  8% (124 of 1517) |

 15% (233 of 1517) |###                  | Elapsed Time: 0:02:51 ETA:   0:24:44Keyword arguments {'add_prefix_space': True} not recognized.
 15% (234 of 1517) |###                  | Elapsed Time: 0:02:51 ETA:   0:23:32Keyword arguments {'add_prefix_space': True} not recognized.
 15% (235 of 1517) |###                  | Elapsed Time: 0:02:52 ETA:   0:17:42Keyword arguments {'add_prefix_space': True} not recognized.
 15% (236 of 1517) |###                  | Elapsed Time: 0:02:52 ETA:   0:16:44Keyword arguments {'add_prefix_space': True} not recognized.
 15% (237 of 1517) |###                  | Elapsed Time: 0:02:53 ETA:   0:11:30Keyword arguments {'add_prefix_space': True} not recognized.
 15% (238 of 1517) |###                  | Elapsed Time: 0:02:53 ETA:   0:11:01Keyword arguments {'add_prefix_space': True} not recognized.
 15% (239 of 1517) |###                  | Elapsed Time: 0:02:55 ETA:   0:16:39Keyword arguments {'add_prefix_space': True} not recognized.
 15% (240 of 1517) |

 23% (350 of 1517) |####                 | Elapsed Time: 0:04:15 ETA:   0:07:52Keyword arguments {'add_prefix_space': True} not recognized.
 23% (351 of 1517) |####                 | Elapsed Time: 0:04:16 ETA:   0:09:54Keyword arguments {'add_prefix_space': True} not recognized.
 23% (352 of 1517) |####                 | Elapsed Time: 0:04:16 ETA:   0:09:00Keyword arguments {'add_prefix_space': True} not recognized.
 23% (353 of 1517) |####                 | Elapsed Time: 0:04:16 ETA:   0:10:01Keyword arguments {'add_prefix_space': True} not recognized.
 23% (354 of 1517) |####                 | Elapsed Time: 0:04:18 ETA:   0:20:42Keyword arguments {'add_prefix_space': True} not recognized.
 23% (355 of 1517) |####                 | Elapsed Time: 0:04:19 ETA:   0:19:33Keyword arguments {'add_prefix_space': True} not recognized.
 23% (356 of 1517) |####                 | Elapsed Time: 0:04:19 ETA:   0:17:18Keyword arguments {'add_prefix_space': True} not recognized.
 23% (357 of 1517) |

 30% (467 of 1517) |######               | Elapsed Time: 0:05:32 ETA:   0:17:42Keyword arguments {'add_prefix_space': True} not recognized.
 30% (468 of 1517) |######               | Elapsed Time: 0:05:33 ETA:   0:18:42Keyword arguments {'add_prefix_space': True} not recognized.
 30% (469 of 1517) |######               | Elapsed Time: 0:05:33 ETA:   0:13:33Keyword arguments {'add_prefix_space': True} not recognized.
 30% (470 of 1517) |######               | Elapsed Time: 0:05:33 ETA:   0:11:24Keyword arguments {'add_prefix_space': True} not recognized.
 31% (471 of 1517) |######               | Elapsed Time: 0:05:34 ETA:   0:10:35Keyword arguments {'add_prefix_space': True} not recognized.
 31% (472 of 1517) |######               | Elapsed Time: 0:05:35 ETA:   0:08:43Keyword arguments {'add_prefix_space': True} not recognized.
Keyword arguments {'add_prefix_space': True} not recognized.
 31% (474 of 1517) |######               | Elapsed Time: 0:05:35 ETA:   0:09:49Keyword arguments {'

...

 65% (993 of 1517) |#############        | Elapsed Time: 0:11:14 ETA:   0:05:50Keyword arguments {'add_prefix_space': True} not recognized.
 65% (994 of 1517) |#############        | Elapsed Time: 0:11:14 ETA:   0:03:33Keyword arguments {'add_prefix_space': True} not recognized.
 65% (995 of 1517) |#############        | Elapsed Time: 0:11:14 ETA:   0:03:33Keyword arguments {'add_prefix_space': True} not recognized.
 65% (996 of 1517) |#############        | Elapsed Time: 0:11:15 ETA:   0:03:50Keyword arguments {'add_prefix_space': True} not recognized.
 65% (997 of 1517) |#############        | Elapsed Time: 0:11:15 ETA:   0:03:58Keyword arguments {'add_prefix_space': True} not recognized.
 65% (998 of 1517) |#############        | Elapsed Time: 0:11:16 ETA:   0:04:48Keyword arguments {'add_prefix_space': True} not recognized.
 65% (999 of 1517) |#############        | Elapsed Time: 0:11:16 ETA:   0:04:44Keyword arguments {'add_prefix_space': True} not recognized.
 65% (1000 of 1517) 

 73% (1109 of 1517) |##############      | Elapsed Time: 0:12:31 ETA:   0:04:04Keyword arguments {'add_prefix_space': True} not recognized.
 73% (1110 of 1517) |##############      | Elapsed Time: 0:12:31 ETA:   0:05:12Keyword arguments {'add_prefix_space': True} not recognized.
 73% (1111 of 1517) |##############      | Elapsed Time: 0:12:32 ETA:   0:05:37Keyword arguments {'add_prefix_space': True} not recognized.
 73% (1112 of 1517) |##############      | Elapsed Time: 0:12:32 ETA:   0:04:56Keyword arguments {'add_prefix_space': True} not recognized.
 73% (1113 of 1517) |##############      | Elapsed Time: 0:12:33 ETA:   0:03:37Keyword arguments {'add_prefix_space': True} not recognized.
 73% (1114 of 1517) |##############      | Elapsed Time: 0:12:33 ETA:   0:03:11Keyword arguments {'add_prefix_space': True} not recognized.
 73% (1115 of 1517) |##############      | Elapsed Time: 0:12:34 ETA:   0:02:52Keyword arguments {'add_prefix_space': True} not recognized.
 73% (1116 of 1517) 

 80% (1225 of 1517) |################    | Elapsed Time: 0:13:51 ETA:   0:04:55Keyword arguments {'add_prefix_space': True} not recognized.
 80% (1226 of 1517) |################    | Elapsed Time: 0:13:51 ETA:   0:03:58Keyword arguments {'add_prefix_space': True} not recognized.
 80% (1227 of 1517) |################    | Elapsed Time: 0:13:52 ETA:   0:03:23Keyword arguments {'add_prefix_space': True} not recognized.
 80% (1228 of 1517) |################    | Elapsed Time: 0:13:52 ETA:   0:02:44Keyword arguments {'add_prefix_space': True} not recognized.
 81% (1229 of 1517) |################    | Elapsed Time: 0:13:53 ETA:   0:03:47Keyword arguments {'add_prefix_space': True} not recognized.
 81% (1230 of 1517) |################    | Elapsed Time: 0:13:54 ETA:   0:05:04Keyword arguments {'add_prefix_space': True} not recognized.
 81% (1231 of 1517) |################    | Elapsed Time: 0:13:55 ETA:   0:04:48Keyword arguments {'add_prefix_space': True} not recognized.
 81% (1232 of 1517) 

 88% (1341 of 1517) |#################   | Elapsed Time: 0:15:13 ETA:   0:02:14Keyword arguments {'add_prefix_space': True} not recognized.
 88% (1342 of 1517) |#################   | Elapsed Time: 0:15:13 ETA:   0:01:50Keyword arguments {'add_prefix_space': True} not recognized.
 88% (1343 of 1517) |#################   | Elapsed Time: 0:15:14 ETA:   0:02:01Keyword arguments {'add_prefix_space': True} not recognized.
 88% (1344 of 1517) |#################   | Elapsed Time: 0:15:14 ETA:   0:01:52Keyword arguments {'add_prefix_space': True} not recognized.
 88% (1345 of 1517) |#################   | Elapsed Time: 0:15:15 ETA:   0:01:45Keyword arguments {'add_prefix_space': True} not recognized.
 88% (1346 of 1517) |#################   | Elapsed Time: 0:15:15 ETA:   0:01:11Keyword arguments {'add_prefix_space': True} not recognized.
 88% (1347 of 1517) |#################   | Elapsed Time: 0:15:16 ETA:   0:01:23Keyword arguments {'add_prefix_space': True} not recognized.
 88% (1348 of 1517) 

 96% (1457 of 1517) |################### | Elapsed Time: 0:16:30 ETA:   0:01:03Keyword arguments {'add_prefix_space': True} not recognized.
 96% (1458 of 1517) |################### | Elapsed Time: 0:16:30 ETA:   0:00:48Keyword arguments {'add_prefix_space': True} not recognized.
 96% (1459 of 1517) |################### | Elapsed Time: 0:16:31 ETA:   0:00:34Keyword arguments {'add_prefix_space': True} not recognized.
 96% (1460 of 1517) |################### | Elapsed Time: 0:16:31 ETA:   0:00:33Keyword arguments {'add_prefix_space': True} not recognized.
 96% (1461 of 1517) |################### | Elapsed Time: 0:16:32 ETA:   0:00:24Keyword arguments {'add_prefix_space': True} not recognized.
 96% (1462 of 1517) |################### | Elapsed Time: 0:16:32 ETA:   0:00:23Keyword arguments {'add_prefix_space': True} not recognized.
 96% (1463 of 1517) |################### | Elapsed Time: 0:16:33 ETA:   0:00:27Keyword arguments {'add_prefix_space': True} not recognized.
 96% (1464 of 1517) 